In [ ]:
import pandas as pd
import numpy as np

PATH_RAW = '../data/raw/D.SDA.PDA.005.CAT.202511.csv'
PATH_PROCESSED = '../data/processed/CAT_Novembro_Final_BI.csv'

def pipeline_limpeza_cat():
    """
    Executa o processo de ETL para a base de acidentes de trabalho (CAT).
    Trata erros de parênteses, nomes cortados e padroniza para o Power BI.
    """
    print("Iniciando processamento da base CAT...")
    

    try:
        df = pd.read_csv(PATH_RAW, sep=';', encoding='latin1')
    except FileNotFoundError:
        print(f"Erro: O arquivo não foi encontrado em {PATH_RAW}")
        return

    cols_mapping = {
        'Agente  Causador  Acidente': 'Agente',
        'Data Acidente': 'Data_Acidente',
        'CNAE2.0 Empregador.1': 'Setor_Economico',
        'Natureza da Lesão': 'Lesao',
        'Parte Corpo Atingida': 'Parte_Corpo',
        'Sexo': 'Sexo',
        'Tipo do Acidente': 'Tipo_Acidente',
        'UF Munic. Empregador': 'UF',
        'Data Nascimento': 'Data_Nascimento'
    }

    
    df_clean = df[list(cols_mapping.keys())].rename(columns=cols_mapping).copy()


    cols_to_format = ['Agente', 'Setor_Economico', 'Lesao', 'Parte_Corpo', 'Sexo', 'Tipo_Acidente', 'UF']
    for col in cols_to_format:
        df_clean[col] = df_clean[col].astype(str).str.strip().str.title()

    mapeamento_limpeza = {
        'Mão (Exceto Punho Ou': 'Mão',
        'Pé (Exceto Artelhos)': 'Pé',
        'Braço (Entre O Punho': 'Braço',
        'Perna (Do Tornozelo,': 'Perna',
        'Perna (Entre O Torno': 'Perna',
        'Doença, Nic': 'Doença Não Informada',
        'Nic': 'Não Informado',
        '{Ñ Class}': 'Não Informado',
        'Zerado': 'Não Informado',
        'Articulação Do Tornozelo': 'Tornozelo',
        'Atividade De Atendi': 'Atendimento Hospitalar',
        'Fabricacao De Alimento': 'Fabricação de Alimentos',
        'Fabricacao De Produt': 'Fabricação de Produtos Químicos',
        'Transporte Rodoviari': 'Transporte Rodoviário'
    }
    
    
    df_clean = df_clean.replace(mapeamento_limpeza, regex=False)

    
    df_clean['Data_Acidente'] = pd.to_datetime(df_clean['Data_Acidente'], format='%d/%m/%Y', errors='coerce')
    df_clean['Data_Nascimento'] = pd.to_datetime(df_clean['Data_Nascimento'], format='%d/%m/%Y', errors='coerce')

    df_clean['Idade'] = (df_clean['Data_Acidente'] - df_clean['Data_Nascimento']).dt.days // 365
    
    
    df_clean.loc[(df_clean['Idade'] < 14) | (df_clean['Idade'] > 100), 'Idade'] = np.nan

    
    df_clean.to_csv(PATH_PROCESSED, index=False, encoding='utf-8-sig')
    
    print("-" * 30)
    print(f"Pipeline finalizado com sucesso!")
    print(f"Arquivo salvo em: {PATH_PROCESSED}")
    print(f"Total de registros: {len(df_clean)}")
    print("-" * 30)

if __name__ == "__main__":
    pipeline_limpeza_cat()